# PyOPV

## Installation

## Example usage

### Importing

In [1]:
import pyopv

## Getting latest DICOM Standards

In [ ]:
from pyopv import get_dicom_standard # if you run this cell, you will get the latest SAP DICOM standard
get_dicom_standard() # if you run this cell, you will get the latest SAP DICOM standard and writes in a csv file in the same directory as this notebook

### Checking missing tags

A single file

In [ ]:
file_path = 'sample_files/SD1031_20230914_OD_OPV.dcm'  # add file path here e.g. data/MY_OPV_FILE.dcm

m_opvdicom = pyopv.read_dicom(file_path)
missing_count, missingtag_df = m_opvdicom.check_missing_tags()
missingtag_df.head(5)

Entire directory

In [ ]:
m_opvdicoms = pyopv.read_dicom_directory('sample_files', file_extension='dcm')
missingtags_df = m_opvdicoms.check_missing_tags()
missingtags_df.head(5)

## Converting OPV DICOM to Pandas DataFrame

*#* This snippet converts the entire dicom file into a very long pandas DataFrame

In [ ]:

from pyopv import dicom_to_dataframe

file_path = 'sample_files/SD1031_20230914_OD_OPV.dcm'  # add file path here e.g. data/MY_OPV_FILE.dcm
dicom_df = dicom_to_dataframe(file_path)
dicom_df.head(3)


# JSON conversion
* this snippet will be updated in future versions to accomodate FHIR IG because that IG is not currently available. 

In [ ]:
import pydicom

file_path = 'sample_files/SD1031_20230914_OD_OPV.dcm' # add file path here e.g. data/MY_OPV_FILE.dcm
ds = pydicom.dcmread(file_path)
json = ds.to_json_dict()
json['00080080']['Value']

In [ ]:
# get nested values
json = ds.to_json_dict()
points = json['00240089']['Value'][0]['00240097']
points

### Getting pointwise data

Single file

In [ ]:
m_opvdicom = pyopv.read_dicom('sample_files/SD1031_20230914_OD_OPV.dcm')

pointwise_data = m_opvdicom.pointwise_to_pandas()
pointwise_data.head(5)

Entire directory

In [ ]:
m_opvdicoms = pyopv.read_dicom_directory('/Users/shahinh/Downloads/OneDrive_1_7-2-2024/SD4132_20240610_OPVtest', file_extension='dcm')

pointwise_data, error_df = m_opvdicoms.pointwise_to_pandas()
pointwise_data.head(3)

In [ ]:
import pandas as pd

# Assuming 'pointwise_data' is already defined as a DataFrame
x_coord = pointwise_data['x_coords']
y_coord = pointwise_data['y_coords']

ns_pointwise_data = pointwise_data.copy()

# Mapping to NS system
# Flip x coordinates for left laterality
ns_pointwise_data.loc[ns_pointwise_data['laterality'] == 'L', 'x_coords'] = -1 * ns_pointwise_data['x_coords']

# Function to map x coordinates to NS system
def map_ns_x(row):
    x_coord = int(row['x_coords'])
    if x_coord > 0:
        return f'T{abs(x_coord)}_'
    else:
        return f'N{abs(x_coord)}_'

# Apply the function to create the 'ns' column
ns_pointwise_data['ns'] = ns_pointwise_data.apply(map_ns_x, axis=1)

# Function to map y coordinates to IS system
def map_ns_y(y):
    y_coord = int(y)
    if y_coord < 0:
        return f'I{abs(y_coord)}'
    else:
        return f'S{y_coord}'

# Apply the function to append y coordinates to the 'ns' column
ns_pointwise_data['ns'] = ns_pointwise_data['ns'] + ns_pointwise_data['y_coords'].apply(map_ns_y)

# Display the updated DataFrame
ns_pointwise_data.head(3)


In [ ]:
# Dictionary mapping original column names to Humphrey Visual Field equivalents
opv_hfa_dict = {
    'sensitivity_values': 'threshold',
    'age_corrected_sensitivity_deviation_values': 'total_deviation',
    'age_corrected_sensitivity_deviation_probability_values': 'total_deviation_probability',
    'generalized_defect_corrected_sensitivity_deviation_flag': 'generalized_defect_threshold_deviation_flag',
    'generalized_defect_corrected_sensitivity_values': 'pattern_deviation',
    'generalized_defect_corrected_sensitivity_probability_values': 'pattern_deviation_probability',
    'ns': 'nasal_temporal_superior_inferior'
}

# Assuming 'pointwise_data' is your DataFrame, you can rename the columns using this dictionary
ns_pointwise_data.rename(columns=opv_hfa_dict, inplace=True)

# Display the modified DataFrame to ensure the changes are correct
ns_pointwise_data.head(3)


In [ ]:
# mapping humphrey equivalent to ucsd equivalent
hfa_ucsd_dict = {
    'threshold': 'Thr',
    'total_deviation': 'TD',
    'total_deviation_probability': 'TDP',
    'generalized_defect_threshold_deviation_flag': 'generalized_defect_corrected_sensitivity_deviation_flag',
    'pattern_deviation': 'PD',
    'pattern_deviation_probability': 'PDP',
    'nasal_temporal_superior_inferior': 'ns'
}

# Assuming 'pointwise_data' is your DataFrame, you can rename the columns using this dictionary
ns_pointwise_data.rename(columns=hfa_ucsd_dict, inplace=True)

# Display the modified DataFrame to ensure the changes are correct
ns_pointwise_data.head(3)

Writing pointwise data into JSON
* Single file

In [ ]:
from pyopv import pointwise_to_nested_json
file_path = 'sample_files/SD1031_20230914_OD_OPV.dcm' # add file path here e.g. data/MY_OPV_FILE.dcm
nested_json = pointwise_to_nested_json(file_path)
nested_json['SD1031']['R']['1.2.276.0.75.2.5.80.25.3.231115115903087.345048637379.415558310'][0]

# Example conversion of a JSON resource to pandas DataFrame

In [ ]:
import pandas as pd
pd.json_normalize(nested_json['SD1031']['R']['1.2.276.0.75.2.5.80.25.3.231115115903087.345048637379.415558310'], sep='_').head(3)

* Bulk data

In [ ]:

from pyopv import opvdicoms_pointwise_to_nested_json 

directory_path = 'sample_files' # add directory path here e.g. data

# Run the function and get the nested JSON
bulk_pointwise_nested_json = opvdicoms_pointwise_to_nested_json(directory_path)

# convert the nested JSON so that the keys are the patient IDs


bulk_pointwise_nested_json['SD1031']['R']['1.2.276.0.75.2.5.80.25.3.221018115130492.345048637379.430073977'][0]

In [ ]:
import pandas as pd
# pd.normalize the nested JSON
bulk_pointwise_nested_json['503286']['L']['2.25.131336704941307872492978734319609842773'][0]

In [ ]:
pd.json_normalize(bulk_pointwise_nested_json['503286']['L']['2.25.131336704941307872492978734319609842773']).head(3)